# Power of Spark w/ Instacart Dataset

Instacart recently released over 30 million transactions from their online grocery store. Details of the release along with the dataset can be found [here](https://www.kaggle.com/c/instacart-market-basket-analysis).

**Citation**  
“The Instacart Online Grocery Shopping Dataset 2017”, Accessed from https://www.instacart.com/datasets/grocery-shopping-2017 on June 5th, 2017

### Step 1: Access the data from the above link and upload to DSX. 

Note that the data is reasonably large (>500mb). Be sure to leave some time for the upload.

In [1]:
spark

### Step 2: Import the transactions list (order_products__prior.csv) and product names/descriptions (products.csv)

In [2]:
from pyspark.sql import SparkSession

# @hidden_cell
# This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_efcfdf24587941948e423d7c9fdd8ff6(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', '694a746300614629be7bdcfa60142623')
    hconf.set(prefix + '.username', '9e80f89e74fe4f1fbe962bb1cac03ab9')
    hconf.set(prefix + '.password', 'OqOVHe-.S,r7UX3~')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_efcfdf24587941948e423d7c9fdd8ff6(name)

spark = SparkSession.builder.getOrCreate()

departments_df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load('swift://InstaCart.' + name + '/departments.csv')
departments_df.take(5)

[Row(department_id=u'1', department=u'frozen'),
 Row(department_id=u'2', department=u'other'),
 Row(department_id=u'3', department=u'bakery'),
 Row(department_id=u'4', department=u'produce'),
 Row(department_id=u'5', department=u'alcohol')]

In [14]:
products_df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load('swift://InstaCart.' + name + '/products.csv')
products_df.take(5)

[Row(product_id=u'1', product_name=u'Chocolate Sandwich Cookies', aisle_id=u'61', department_id=u'19'),
 Row(product_id=u'2', product_name=u'All-Seasons Salt', aisle_id=u'104', department_id=u'13'),
 Row(product_id=u'3', product_name=u'Robust Golden Unsweetened Oolong Tea', aisle_id=u'94', department_id=u'7'),
 Row(product_id=u'4', product_name=u'Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce', aisle_id=u'38', department_id=u'1'),
 Row(product_id=u'5', product_name=u'Green Chile Anytime Sauce', aisle_id=u'5', department_id=u'13')]

In [3]:
order_products_df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load('swift://InstaCart.' + name + '/order_products__prior.csv')
order_products_df.take(5)


[Row(order_id=u'2', product_id=u'33120', add_to_cart_order=u'1', reordered=u'1'),
 Row(order_id=u'2', product_id=u'28985', add_to_cart_order=u'2', reordered=u'1'),
 Row(order_id=u'2', product_id=u'9327', add_to_cart_order=u'3', reordered=u'0'),
 Row(order_id=u'2', product_id=u'45918', add_to_cart_order=u'4', reordered=u'1'),
 Row(order_id=u'2', product_id=u'30035', add_to_cart_order=u'5', reordered=u'0')]

### Step 3: Look at the data!

In [4]:
order_products_df.show()

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       2|     33120|                1|        1|
|       2|     28985|                2|        1|
|       2|      9327|                3|        0|
|       2|     45918|                4|        1|
|       2|     30035|                5|        0|
|       2|     17794|                6|        1|
|       2|     40141|                7|        1|
|       2|      1819|                8|        1|
|       2|     43668|                9|        0|
|       3|     33754|                1|        1|
|       3|     24838|                2|        1|
|       3|     17704|                3|        1|
|       3|     21903|                4|        1|
|       3|     17668|                5|        1|
|       3|     46667|                6|        1|
|       3|     17461|                7|        1|
|       3|     32665|                8|        1|


### Establish some simple benchmarks for count and groupby

In [5]:
%time order_products_df.count()

CPU times: user 10.8 ms, sys: 5.19 ms, total: 16 ms
Wall time: 24.2 s


32434489

### What are the most purchased products in the dataset? 

In [16]:
order_products_df.groupBy("product_id").count().join(products_df, on="product_id").show()

+----------+-----+--------------------+--------+-------------+
|product_id|count|        product_name|aisle_id|department_id|
+----------+-----+--------------------+--------+-------------+
|     35004| 8200|Macaroni Shells &...|       4|            9|
|     39581| 8987|French Vanilla Cr...|      53|           16|
|      8433|   71|        Pronto Penne|     131|            9|
|     11078| 3070|     Macaroni Elbows|     131|            9|
|      7762| 1652|   Vodka Pasta Sauce|       9|            9|
|     38223|  750|Stage 1 Just Swee...|      92|           18|
|     28473| 1479|Wavy Original Pot...|     107|           19|
|     28135|   55|Deli-Sliced Roast...|      81|           15|
|     15269| 2874|Natural Medium Ch...|      21|           16|
|     47140|  182|7OZ SAUSAGE GRAVY...|      52|            1|
|     16974| 2773|Sea Salt Brown Ri...|     107|           19|
|       691| 3151|Organic Promise S...|     121|           14|
|     49079| 1063|Special K Vanilla...|     121|       

In [17]:
# The above didn't quite do it... we need to order.
grouped_df = order_products_df.groupBy("product_id").count().join(products_df, on="product_id").orderBy("count", ascending=False)
grouped_df.select("product_id", "product_name", "count").show()

+----------+--------------------+------+
|product_id|        product_name| count|
+----------+--------------------+------+
|     24852|              Banana|472565|
|     13176|Bag of Organic Ba...|379450|
|     21137|Organic Strawberries|264683|
|     21903|Organic Baby Spinach|241921|
|     47209|Organic Hass Avocado|213584|
|     47766|     Organic Avocado|176815|
|     47626|         Large Lemon|152657|
|     16797|        Strawberries|142951|
|     26209|               Limes|140627|
|     27845|  Organic Whole Milk|137905|
|     27966| Organic Raspberries|137057|
|     22935|Organic Yellow Onion|113426|
|     24964|      Organic Garlic|109778|
|     45007|    Organic Zucchini|104823|
|     39275| Organic Blueberries|100060|
|     49683|      Cucumber Kirby| 97315|
|     28204|  Organic Fuji Apple| 89632|
|      5876|       Organic Lemon| 87746|
|      8277|Apple Honeycrisp ...| 85020|
|     40706|Organic Grape Tom...| 84255|
+----------+--------------------+------+
only showing top

### Performance Optimization

#### Add `order_products_df` and `products_df` as spark views (SQL tables)

In [18]:
order_products_df.createTempView("order_products")
products_df.createTempView("products")

#### Cache these tables

In [19]:
spark.catalog.cacheTable("order_products")
spark.catalog.cacheTable("products")

#### Point our dataframes towards these tables

In [21]:
order_products_df = spark.table("order_products")
products_df = spark.table("products")

### That was fast... lazy evaluation! 

In [25]:
order_products_df.count()
products_df.count()

49688

### Now, Lets try this again...

In [26]:
order_products_df.count()

32434489

In [27]:
products_df.count()

49688

In [28]:
# Run our large group by command again
grouped_df = order_products_df.groupBy("product_id").count().join(products_df, on="product_id").orderBy("count", ascending=False)
grouped_df.select("product_id", "product_name", "count").show()

+----------+--------------------+------+
|product_id|        product_name| count|
+----------+--------------------+------+
|     24852|              Banana|472565|
|     13176|Bag of Organic Ba...|379450|
|     21137|Organic Strawberries|264683|
|     21903|Organic Baby Spinach|241921|
|     47209|Organic Hass Avocado|213584|
|     47766|     Organic Avocado|176815|
|     47626|         Large Lemon|152657|
|     16797|        Strawberries|142951|
|     26209|               Limes|140627|
|     27845|  Organic Whole Milk|137905|
|     27966| Organic Raspberries|137057|
|     22935|Organic Yellow Onion|113426|
|     24964|      Organic Garlic|109778|
|     45007|    Organic Zucchini|104823|
|     39275| Organic Blueberries|100060|
|     49683|      Cucumber Kirby| 97315|
|     28204|  Organic Fuji Apple| 89632|
|      5876|       Organic Lemon| 87746|
|      8277|Apple Honeycrisp ...| 85020|
|     40706|Organic Grape Tom...| 84255|
+----------+--------------------+------+
only showing top

## MUCH BETTER :)